## acetate and glycerol cross-feeding in glucose-limited environments in e. coli
- E. coli iJO1366
- glucose-acetate and glucose-glycerol interactions
- Acetate fluxes controlled by Pta-AckA pathway or acetyl-coA synthetase (forms acetyl-CoA)

## Expected results
- E. coli will co-consume glucose and acetate under glucose excess (Enjalbert et. al)
- E. coli will consume PRODUCE acetate once glucose is completely consumed (Enjalbert et. al)

## Questions
- How do I figure out which exchange reactions are associated with my carbon sources?

In [1]:
from copy import deepcopy
import cobra
from process_bigraph import Composite
from process_bigraph import ProcessTypes
from process_bigraph.emitter import gather_emitter_results

from cdFBA import register_types
from cdFBA.processes.dfba import dFBA, UpdateEnvironment, StaticConcentration, Injector, WaveFunction

from cdFBA.utils import make_cdfba_composite, get_injector_spec, get_wave_spec, get_static_spec, set_concentration, set_kinetics

from pathlib import Path
from cobra.io import load_json_model, save_json_model, load_matlab_model, save_matlab_model, read_sbml_model, write_sbml_model
import logging

from matplotlib import pyplot as plt
from pprint import pprint

import cProfile
import pstats
import io


In [2]:
#inspect E. coli model
e_coli_lac = cobra.io.load_model("iJO1366")
e_coli_glyc_ex = e_coli_lac.copy()

In [3]:
def get_all_met_ids(model, met_name):
    """Gets all metabolites ids associated with a user provided string"""
    all_matches = []
    for met in model.metabolites:
        if met_name in met.id:
            all_matches.append(met.id)
    return all_matches


In [4]:
def make_met_and_rxn_dict(model, met_name):
    """Makes a dictionary with metabolite ids as keys and rxns as values"""
    dict = {}
    all_matches = get_all_met_ids(model, met_name)
    for met in all_matches:
        rxn = model.metabolites.get_by_id(met).reactions
        for rxn in rxn:
            dict[met] = rxn.id
    return dict

In [5]:
glucose_rxn_dict = make_met_and_rxn_dict(e_coli_lac, "glc") # inspect to find glucose exchange reactions

In [6]:
acetate_rxn_dict = make_met_and_rxn_dict(e_coli_lac, "ac") # inspect to find acetate exchange reactions

In [7]:
def make_met_and_genes_dict(model, met_name):
    """Makes a dictionary with metabolite ids as keys and gene ids as values"""
    dict = {}
    all_matches = get_all_met_ids(model, met_name)
    for met_id in all_matches:
        met = model.metabolites.get_by_id(met_id)
        genes = []
        for rxn in met.reactions:
            for gene in rxn.genes:
                genes.append(gene.id)
        dict[met_id] = genes
    return dict

In [8]:
glucose_genes_dict = make_met_and_genes_dict(e_coli_lac, "glc")

In [9]:
acetate_genes_dict = make_met_and_genes_dict(e_coli_lac, "ac")

In [10]:
glycerol_genes_dict = make_met_and_genes_dict(e_coli_lac, "gly")

In [11]:
glyceral_reactions_dict = make_met_and_rxn_dict(e_coli_lac, "gly")

In [12]:
pprint(glyceral_reactions_dict)

{'2pglyc_c': 'PGLYCP',
 'bglycogen_c': 'GLCP2',
 'cgly_c': 'AMPTASECG',
 'cgly_e': 'EX_cgly_e',
 'cgly_p': 'GTHRDHpp',
 'dhgly_c': 'TYRL',
 'gly_c': 'GLYt4pp',
 'gly_e': 'GLYtex',
 'gly_p': 'GLYt2pp_copy1',
 'glyald_c': 'ALCD19',
 'glyald_e': 'GLYALDtex',
 'glyald_p': 'GLYALDtex',
 'glyb_c': 'GLYBabcpp',
 'glyb_e': 'EX_glyb_e',
 'glyb_p': 'GLYBabcpp',
 'glyc2p_c': 'G2PP',
 'glyc2p_e': 'GLYC2Ptex',
 'glyc2p_p': 'GLYC2Pabcpp',
 'glyc3p_c': 'G3PD5',
 'glyc3p_e': 'GLYC3Ptex',
 'glyc3p_p': 'GLYC3Pabcpp',
 'glyc__R_c': 'TRSARr',
 'glyc__R_e': 'EX_glyc__R_e',
 'glyc__R_p': 'GLYCAt2rpp',
 'glyc_c': 'G2PP',
 'glyc_e': 'EX_glyc_e',
 'glyc_p': 'CLPNS160pp',
 'glyclt_c': 'PGLYCP',
 'glyclt_e': 'EX_glyclt_e',
 'glyclt_p': 'GLYCLTt2rpp',
 'glycogen_c': 'GLDBRAN2',
 'glytrna_c': 'GLYTRS',
 'man6pglyc_c': 'MANGLYCptspp',
 'manglyc_e': 'MANGLYCtex',
 'manglyc_p': 'MANGLYCtex',
 'progly_c': 'AMPTASEPG',
 'progly_e': 'PROGLYtex',
 'progly_p': 'PROGLYtex',
 'trnagly_c': 'GLYTRS',
 'urdglyc_c': 'ALLTAMH'}


In [13]:
pprint(glycerol_genes_dict)

{'2pglyc_c': ['b3385'],
 'bglycogen_c': ['b3432', 'b3431', 'b3428', 'b3417'],
 'cgly_c': ['b3540',
            'b3541',
            'b3542',
            'b3543',
            'b3544',
            'b4260',
            'b0932',
            'b2523',
            'b0237'],
 'cgly_e': ['b1377', 'b0929', 'b0241', 'b2215'],
 'cgly_p': ['b3540',
            'b3541',
            'b3542',
            'b3543',
            'b3544',
            'b1377',
            'b0929',
            'b0241',
            'b2215',
            'b3447'],
 'dhgly_c': ['b0423', 'b2530', 'b3990', 'b3992', 'b4407', 'b3991'],
 'gly_c': ['b2551',
           'b4260',
           'b0932',
           'b2523',
           'b0237',
           'b2551',
           'b0870',
           'b4005',
           'b2903',
           'b2905',
           'b2904',
           'b0116',
           'b2947',
           'b1059',
           'b4260',
           'b0932',
           'b2523',
           'b0237',
           'b3559',
           'b3560',
    

In [14]:
print(glyceral_reactions_dict["cgly_e"])

EX_cgly_e


In [15]:
print(glycerol_genes_dict["cgly_e"]) # genes associated w/EX_cgly_e

['b1377', 'b0929', 'b0241', 'b2215']


## Design
- Make one that cannot take up lactose
- Make another that cannot take up glycerol
- Glucose abundant environment

In [16]:
# make KOs
with e_coli_glyc_ex:
    e_coli_glyc_ex.genes.b0929.knock_out() # glycerol metabolism
    e_coli_lac.reactions.LCTStex.knock_out() # cannot uptake lactose
    write_sbml_model(e_coli_glyc_ex, "/Vivarium/cdFBA-main/Notebooks/e_coli_glyc_ex_ko.xml")
    write_sbml_model(e_coli_lac, "/Vivarium/cdFBA-main/Notebooks/e_coli_lac_ko.xml")